Want:

main/control loss for:

- GSAE
- GSAE finetune
- GSAE-phys (sweep L0)
- GSAE + SSAE (sweep L0)
- clean loss

In [1]:
import sys
sys.path.append("/root/specialised-SAEs")
from datasets import load_dataset
from transformer_lens import utils, HookedTransformer
import gc
import torch
from sae_lens.jacob.load_sae_from_hf import load_sae_from_hf
from config import DTYPE_MAP
from tqdm import tqdm

In [2]:
torch.set_grad_enabled(False)

In [3]:
DTYPE = "float32"

In [4]:
model = HookedTransformer.from_pretrained("gpt2-small", device="cuda", dtype=DTYPE)

Loaded pretrained model gpt2-small into HookedTransformer


In [5]:
gsae_64 = load_sae_from_hf("jacobcd52/gpt2-gsae", 
                        "expansion=64.safetensors", 
                        "expansion=64_cfg.json",
                        device="cuda",
                        dtype=DTYPE)

ssae_list = [
    load_sae_from_hf("jacobcd52/gpt2-ssae-phys-widegsae",
                    f"l1_coeff={l1_coeff}_expansion=2_control=0.0.safetensors",
                    f"l1_coeff={l1_coeff}_expansion=2_control=0.0_cfg.json",
                    device="cuda",
                    dtype=DTYPE)
    for l1_coeff in [2, 3, 4, 5, 6]
]

gsae_slow_list = [
    load_sae_from_hf("jacobcd52/gpt2-gsae-finetune-phys",
                    f"l1_coeff={l1_coeff}_tokens=819200_lr=1e-05.safetensors",
                    f"l1_coeff={l1_coeff}_tokens=819200_lr=1e-05_cfg.json",
                    device="cuda",
                    dtype=DTYPE)
    for l1_coeff in [1, 10, 20, 30, 45, 55]
]

gsae_medium_list = [
    load_sae_from_hf("jacobcd52/gpt2-gsae-finetune-phys",
                    f"l1_coeff={l1_coeff}_tokens=819200_lr=0.0001.safetensors",
                    f"l1_coeff={l1_coeff}_tokens=819200_lr=0.0001_cfg.json",
                    device="cuda",
                    dtype=DTYPE)
    for l1_coeff in [1, 10, 20, 30, 45, 55]
]

gsae_fast_list = [
    load_sae_from_hf("jacobcd52/gpt2-gsae-finetune-phys",
                    f"l1_coeff={l1_coeff}_tokens=819200_lr=0.001.safetensors",
                    f"l1_coeff={l1_coeff}_tokens=819200_lr=0.001_cfg.json",
                    device="cuda",
                    dtype=DTYPE)
    for l1_coeff in [1, 10, 20, 30, 45, 55]
]

expansion=64.safetensors:   0%|          | 0.00/303M [00:00<?, ?B/s]

GSAE weights file saved as temp_sae/sae_weights.safetensors


expansion=64_cfg.json:   0%|          | 0.00/2.48k [00:00<?, ?B/s]

GSAE cfg file saved as temp_sae/cfg.json
Loading weights into GSAE from temp_sae/sae_weights.safetensors
temp_sae/cfg.json temp_sae/sae_weights.safetensors


(…)ff=2_expansion=2_control=0.0.safetensors:   0%|          | 0.00/85.1M [00:00<?, ?B/s]

...

GSAE cfg file saved as temp_sae/cfg.json
Loading weights into GSAE from temp_sae/sae_weights.safetensors
temp_sae/cfg.json temp_sae/sae_weights.safetensors


(…)ff=55_tokens=819200_lr=0.001.safetensors:   0%|          | 0.00/303M [00:00<?, ?B/s]

GSAE weights file saved as temp_sae/sae_weights.safetensors


(…)coeff=55_tokens=819200_lr=0.001_cfg.json:   0%|          | 0.00/2.56k [00:00<?, ?B/s]

GSAE cfg file saved as temp_sae/cfg.json
Loading weights into GSAE from temp_sae/sae_weights.safetensors
temp_sae/cfg.json temp_sae/sae_weights.safetensors


In [6]:
# get OWT tokens
data = load_dataset("stas/openwebtext-10k", split="train")
tokenized_data = utils.tokenize_and_concatenate(data, model.tokenizer, max_length=256)
tokenized_data = tokenized_data.shuffle(42)
owt_tokens = tokenized_data["tokens"][:20_000].cuda()
print("owt_tokens has shape", owt_tokens.shape)
print("total number of tokens:", int(owt_tokens.numel()//1e6), "million")
print()
# get physics-papers tokens
data = load_dataset("jacobcd52/physics-papers", split="train[:10%]")
# Define a filter function to remove null entries
def remove_null_entries(example):
    return all(value is not None and value != '' for value in example.values())
# Apply the filter to remove null entries
data = data.filter(remove_null_entries)
tokenized_data = utils.tokenize_and_concatenate(data, model.tokenizer, max_length=256)
tokenized_data = tokenized_data.shuffle(42)
phys_tokens = tokenized_data["tokens"][:20_000].cuda()
print("phys_tokens has shape", phys_tokens.shape)
print("total number of tokens:", int(phys_tokens.numel()//1e6), "million")

# clean up
del tokenized_data, data
gc.collect()

owt_tokens has shape torch.Size([20000, 256])
total number of tokens: 5 million

phys_tokens has shape torch.Size([20000, 256])
total number of tokens: 5 million


0

In [7]:
def get_loss(
        sae_list, 
        tokens, 
        num_tokens = 1_000_000, 
        hook_pt = 'blocks.8.hook_resid_pre', 
        batch_size=64
        ):
    
    # define hook fn to patch in SAE reconstructions, as well as cache the L0
    l0_dic = {}
    def patch_hook(act, hook):              
        # option to run the model clean
        if sae_list == "clean":
            l0_dic[0] = 0
            return act
        # patch in SAE reconstructions
        else:
            l0 = 0
            out = torch.zeros_like(act)
            for sae in sae_list:
                feature_acts = sae.encode_fn(act)
                l0 += (feature_acts > 0).to(DTYPE_MAP[DTYPE]).sum(dim=-1).mean().item()
                out += sae.decode(feature_acts)

        # save l0 to dictionary and return the output to patch
        l0_dic[0] = l0
        return out
    
    # initialise running variables
    total_l0 = 0
    total_loss = 0

    num_batches = num_tokens // (tokens.shape[1] * batch_size)

    for b in range(num_batches):
        # get batch
        batch = tokens[b*batch_size:(b+1)*batch_size]
        total_loss += model.run_with_hooks(
            batch,
            return_type="loss",
            fwd_hooks = [(hook_pt, patch_hook)]
        ).item()

        total_l0 += l0_dic[0]

        # print(f"batch {b} loss: {total_loss / (b+1)}   L0: {total_l0 / (b+1)}")
        
    return total_loss / num_batches, total_l0 / num_batches

In [8]:
# Get baseline losses, where we don't need to sweep
clean_owt_loss, _ = get_loss("clean", owt_tokens)
clean_phys_loss, _ = get_loss("clean", phys_tokens)
gsae_owt_loss, gsae_owt_l0 = get_loss([gsae_64], owt_tokens)
gsae_phys_loss, gsae_phys_l0 = get_loss([gsae_64], phys_tokens)

print("clean OWT loss = ", clean_owt_loss)
print("clean physics loss = ", clean_phys_loss)
print()
print("gsae OWT loss = ", gsae_owt_loss)
print("gsae physics loss = ", gsae_phys_loss)
print()
print("gsae OWT L0 = ", int(gsae_owt_l0))
print("gsae physics L0 = ", int(gsae_phys_l0))

clean OWT loss =  3.4051552326952823
clean physics loss =  3.8703098805224307

gsae OWT loss =  3.7742148422804034
gsae physics loss =  4.485767411403969

gsae OWT L0 =  78
gsae physics L0 =  97


In [9]:
def get_l0_and_scores(sae_list_list, tokens, ceiling_loss, clean_loss, num_tokens = 1_000_000):
    score_list = []
    l0_list = []
    for sae_list in tqdm(sae_list_list):
        loss, l0 = get_loss(sae_list, tokens, num_tokens=num_tokens)
        score = (ceiling_loss - loss) / (ceiling_loss - clean_loss)
        score_list.append(score)
        l0_list.append(l0)
        # print("score = ", score)
    return l0_list, score_list

In [10]:
# get pareto data for SSAEs
ssae_list_list = [[ssae, gsae_64] for ssae in ssae_list]
print("SSAE OWT scores:\n")
ssae_owt_l0, ssae_owt_scores = get_l0_and_scores(ssae_list_list, owt_tokens, gsae_owt_loss, clean_owt_loss)
print("\nSSAE phys scores\n")
ssae_phys_l0, ssae_phys_scores = get_l0_and_scores(ssae_list_list, phys_tokens, gsae_phys_loss, clean_phys_loss)

# get pareto data for finetuned GSAEs
gsae_fast_list_list = [[gsae] for gsae in gsae_fast_list]
print("\n\nfast OWT scores:\n")
gsae_fast_owt_l0, gsae_fast_owt_scores = get_l0_and_scores(gsae_fast_list_list, owt_tokens, gsae_owt_loss, clean_owt_loss)
print("\nfast phys scores\n")
gsae_fast_phys_l0, gsae_fast_phys_scores = get_l0_and_scores(gsae_fast_list_list, phys_tokens, gsae_phys_loss, clean_phys_loss)

gsae_medium_list_list = [[gsae] for gsae in gsae_medium_list]
print("\n\nmedium OWT scores:\n")
gsae_medium_owt_l0, gsae_medium_owt_scores = get_l0_and_scores(gsae_medium_list_list, owt_tokens, gsae_owt_loss, clean_owt_loss)
print("\nmedium phys scores\n")
gsae_medium_phys_l0, gsae_medium_phys_scores = get_l0_and_scores(gsae_medium_list_list, phys_tokens, gsae_phys_loss, clean_phys_loss)

gsae_slow_list_list = [[gsae] for gsae in gsae_slow_list]
print("\n\nslow OWT scores:\n")
gsae_slow_owt_l0, gsae_slow_owt_scores = get_l0_and_scores(gsae_slow_list_list, owt_tokens, gsae_owt_loss, clean_owt_loss)
print("\nslow phys scores\n")
gsae_slow_phys_l0, gsae_slow_phys_scores = get_l0_and_scores(gsae_slow_list_list, phys_tokens, gsae_phys_loss, clean_phys_loss)

SSAE OWT scores:



100%|██████████| 5/5 [03:26<00:00, 41.27s/it]



SSAE phys scores



100%|██████████| 5/5 [03:26<00:00, 41.33s/it]




fast OWT scores:



...

100%|██████████| 6/6 [04:03<00:00, 40.64s/it]




slow OWT scores:



100%|██████████| 6/6 [04:03<00:00, 40.63s/it]



slow phys scores



100%|██████████| 6/6 [04:03<00:00, 40.63s/it]


In [13]:
import pandas as pd
import plotly.express as px

# Create a DataFrame for each set of data
df_ssae = pd.DataFrame({'l0': ssae_owt_l0, 'Scores': ssae_owt_scores, 'Type': 'SSAE'})
df_gsae_fast = pd.DataFrame({'l0': gsae_fast_owt_l0, 'Scores': gsae_fast_owt_scores, 'Type': 'GSAE Fast'})
df_gsae_slow = pd.DataFrame({'l0': gsae_slow_owt_l0, 'Scores': gsae_slow_owt_scores, 'Type': 'GSAE Slow'})
df_gsae_medium = pd.DataFrame({'l0': gsae_medium_owt_l0, 'Scores': gsae_medium_owt_scores, 'Type': 'GSAE Medium'})

# Combine the DataFrames
df = pd.concat([df_ssae, df_gsae_fast, df_gsae_slow, df_gsae_medium])

# Create a line plot
fig = px.line(df, x='l0', y='Scores', color='Type', labels={'l0': 'l0', 'Scores': 'Scores'}, title='OWT Pareto Frontiers')

# Show the plot
fig.show()

/usr/local/lib/python3.10/dist-packages/plotly/express/_core.py:2065: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.



In [14]:
import pandas as pd
import plotly.express as px

# Create a DataFrame for each set of data
df_ssae = pd.DataFrame({'l0': ssae_phys_l0, 'Scores': ssae_phys_scores, 'Type': 'SSAE'})
df_gsae_fast = pd.DataFrame({'l0': gsae_fast_phys_l0, 'Scores': gsae_fast_phys_scores, 'Type': 'GSAE Fast'})
df_gsae_slow = pd.DataFrame({'l0': gsae_slow_phys_l0, 'Scores': gsae_slow_phys_scores, 'Type': 'GSAE Slow'})
df_gsae_medium = pd.DataFrame({'l0': gsae_medium_phys_l0, 'Scores': gsae_medium_phys_scores, 'Type': 'GSAE Medium'})

# Combine the DataFrames
df = pd.concat([df_ssae, df_gsae_fast, df_gsae_slow, df_gsae_medium])

# Create a line plot
fig = px.line(df, x='l0', y='Scores', color='Type', labels={'l0': 'l0', 'Scores': 'Scores'}, title='Phys Pareto Frontiers')

# Show the plot
fig.show()

/usr/local/lib/python3.10/dist-packages/plotly/express/_core.py:2065: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.



: 